# **Car Detection From CCTV**

# **Import Libraries**

In [2]:
import cv2
import numpy as np


# **Loading Video .....**

In [10]:
video_path = r"C:\Users\Ahmad\Downloads\Traffic IP Camera video.mp4"
cap = cv2.VideoCapture(video_path)

# **Apply Open CV Attributes**

In [11]:
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

car_positions = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (800, 600))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    fgmask = fgbg.apply(gray)

    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    new_positions = {}

    for contour in contours:
        if cv2.contourArea(contour) > 800:  
            x, y, w, h = cv2.boundingRect(contour)

            center = (x + w // 2, y + h // 2)
            new_positions[center] = (x, y, w, h)

            if center in car_positions:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  
                cv2.putText(frame, "Stopped Car", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            else:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  
                cv2.putText(frame, "Moving Car", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    car_positions = new_positions

    cv2.imshow("Car Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()